<a href="https://colab.research.google.com/github/ewaterland/Wi-Fi_Data_Pre-processing/blob/main/Wi_Fi_Data_Pre_processing_v3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

수지가 전처리 하는 방식

### import

In [1]:
import os
import csv
import pandas as pd
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ★ 데이터를 구별할 수 있는 **위치 정보, 데이터 수집 날짜, 버전** 입력 ★

In [3]:
temp = "8th"

In [4]:
date = "230916"

In [5]:
ver = "v005"

In [6]:
buffer = "5"

In [7]:
#sign = "/pos"
sign = "/neg"
#sign = ""

#sep = "/구간1"
#sep = "/구간2"
sep = ""

In [8]:
# Offline: 0 / Online: 1

data = 0

print("data: %d" % data)

path = "/content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/"
path_temp = path + date + " " + temp + "/"         # ex. 230703 MIPL/

# 0: Offline, 1: Online
if data == 0:
  offline_path = path_temp + "Offline Data/"
  path = offline_path
  data_type = "_offline"
elif data == 1:
  online_path = path_temp + "Online Data/"
  path = online_path
  data_type = "_online"

data: 0


### Number of Mac Address

In [9]:
nom = 20

nom_values = list(range(2, 21))  # 2부터 20까지의 nom 값 리스트. (시작, 종료, 간격)

# Data Pre-processing

## Wi-Fi 수집 데이터 전처리 (여러 개) - 코드 2개

In [12]:
# =================================================================================================================

def process_csv_folder(input_folder_path, output_folder_path, sep=','):
    # Get a list of all files in the input folder
    file_list = os.listdir(input_folder_path)

    # Iterate over each file in the folder
    for file_name in file_list:
        # Construct the input and output file paths
        input_file_path = os.path.join(input_folder_path, file_name)
        output_file_name = os.path.splitext(file_name)[0] + "_MAC " + str(nom) + ".csv"
        output_file_path = os.path.join(output_folder_path, output_file_name)

        file_name = file_name.replace(".csv", "")

        # 파일 이름에서 알파벳 제거 / posx, posy 값을 추출
        pattern = re.compile('[a-zA-Z]')
        # 정규 표현식에 맞는 문자를 공백으로 대체하여 알파벳 문자를 제거
        file_name = re.sub(pattern, '', file_name)

        #posx, posy = re.findall(r'\d+', file_name)[0], re.findall(r'\d+', file_name)[1]
        #posx, posy = file_name.split('.csv')[0].split(',')  # 소수점
        posx, posy = file_name.split(',')  # 소수점

        # Process the CSV file and save the modified output file
        process_csv(input_file_path, output_file_path, posx, posy, sep)

# =================================================================================================================

def process_csv(input_file_path, output_file_path, x, y, sep=','):
    df = pd.read_csv(input_file_path, sep=sep)

    # Extract the MAC addresses from the first row and remove extra decimal points
    mac_addresses = df.columns.str.replace(r'\.\d+', '')

    # Calculate the mean values for each column
    mean_row = df.mean()

    # Create a new DataFrame with the MAC addresses and the mean values
    new_df = pd.DataFrame(columns=mac_addresses)
    new_df.loc[0] = mac_addresses
    new_df.loc[1] = mean_row.values

    # 'Timestamp, BT, BY, BZ, BX' 열을 삭제
    #new_df = new_df.iloc[:, :-5]

    # 평균 낸 RSS 값 선택
    rss_mean = new_df.iloc[1]

    # 평균 낸 RSS 값이 큰 순서로 데이터프레임 정렬
    sorted_df = new_df[rss_mean.sort_values(ascending=False).index]

    # 중복되는 행 삭제
    sorted_df = sorted_df.drop(0)

    # 설정한 AP 개수대로 신호 세기가 센 신호 남기기
    signal_df = sorted_df.iloc[:, 0:nom]

    # 첫 번째 열에 posx, posy 값을 추가
    posx = x
    posy = y
    signal_df.insert(0, 'posx', posx)
    signal_df.insert(1, 'posy', posy)

    make_df = pd.DataFrame(columns=["posx", "posy", "mac", "rss"])

    x = signal_df.iloc[0]["posx"]
    y = signal_df.iloc[0]["posy"]

    print(signal_df)

    # Loop through each MAC address and RSS value
    for j in range(2, nom+2):

      try:
        # Get the MAC address and RSS value
        mac = signal_df.columns[j]
        rss = signal_df.iloc[0][j]
      except:
        print("nom: %d 오류" % nom)

      # Add a new row to the new DataFrame
      make_df = make_df.append({
          "posx": x,
          "posy": y,
          "mac": mac,
          "rss": rss
      }, ignore_index=True)

      make_df.to_csv(output_file_path, sep=sep, index=False, header=True)

# =================================================================================================================

for nom in nom_values:
  #input_folder_path = path + 'Original/'  # 필터 안 쓴 Original 데이터
  #input_folder_path = path + 'Moving Avg Filter_BF ' + buffer + '/Filtered Data/'  # 이동평균필터 적용한 데이터
  input_folder_path = path + 'Moving Avg Filter_BF ' + buffer + sign + '/Filtered Data' + sep + '/'
  #output_folder_path = path + 'Pre-processing ' + ver + '/MAC ' + str(nom) + '/temp/'
  output_folder_path = path + 'Pre-processing ' + ver + sign + sep + '/MAC ' + str(nom) + '/temp/'

  # Create the output folder if it doesn't exist
  if not os.path.exists(output_folder_path):
      os.makedirs(output_folder_path)

  process_csv_folder(input_folder_path, output_folder_path)

<ipython-input-12-aff822e869b0>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  mac_addresses = df.columns.str.replace(r'\.\d+', '')
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  mac_addresses = df.columns.str.replace(r'\.\d+', '')
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: 

  posx posy 00:40:5a:bf:d0:b2 00:40:5a:bf:d0:b3
1    0  -30        -32.484211        -34.133333
  posx posy 00:40:5a:bf:d0:b3 6c:c4:9f:99:e1:f0
1    0  -28        -37.666667        -40.422807
  posx posy 6c:c4:9f:99:e1:f0 6c:c4:9f:99:e1:f2
1    0  -32         -43.07037        -43.171296
  posx posy 6c:c4:9f:9b:3f:30 6c:c4:9f:9b:3f:31
1    0  -26        -41.691176         -41.82549
  posx posy 6c:c4:9f:9b:3f:30 6c:c4:9f:9b:3f:31
1    0  -20         -34.90098        -35.597059


<ipython-input-12-aff822e869b0>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  mac_addresses = df.columns.str.replace(r'\.\d+', '')
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({


  posx posy 6c:c4:9f:9b:3f:30 6c:c4:9f:9b:3f:32
1    0  -22        -37.185294        -37.408824


<ipython-input-12-aff822e869b0>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  mac_addresses = df.columns.str.replace(r'\.\d+', '')
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({


...

<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-i

  posx posy 6c:c4:9f:99:d3:d0 00:40:5a:c0:06:02 00:40:5a:c0:06:03  \
1    0   -2        -43.134314        -45.268627        -46.327451   

  6c:c4:9f:99:d3:d1 6c:c4:9f:99:d3:d2 00:40:5a:c0:58:9a 6c:c4:9f:9b:3f:31  \
1        -48.020588        -48.020588        -48.057843        -52.307843   

  6c:c4:9f:9b:3f:30  ... 6c:c4:9f:99:bb:70 6c:c4:9f:99:bb:72  \
1        -52.754902  ...        -53.395098        -53.406863   

  6c:c4:9f:99:bb:71 00:40:5a:c0:58:a3 00:40:5a:c0:58:9b 06:30:0d:5d:f2:be  \
1        -53.865686        -54.422549        -55.242157             -56.1   

  00:40:5a:c0:58:a1 00:40:5a:c0:58:99 6c:c4:9f:99:d3:c1 6c:c4:9f:90:f7:72  
1        -56.143137        -56.428431        -59.404902        -60.655882  

[1 rows x 22 columns]


<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-i

   posx posy 6c:c4:9f:9a:bf:e0 6c:c4:9f:9a:bf:e2 6c:c4:9f:9a:bf:e1  \
1  1.35  -42        -45.239815        -45.490741        -46.737963   

  00:40:5a:bf:d0:b3 00:30:0d:62:21:26 00:40:5a:bf:d0:b1 00:40:5a:bf:d0:b2  \
1        -50.226852        -50.593519        -50.678704        -50.896296   

  06:30:0d:62:21:26  ... 00:40:5a:c0:58:b3 6c:c4:9f:9a:bf:f2  \
1        -51.217593  ...        -53.412037        -53.873148   

  6c:c4:9f:9a:bf:f1 6c:c4:9f:9a:bf:f3 6c:c4:9f:9a:bf:f0 00:40:5a:c0:58:b2  \
1        -53.988889        -54.011111        -54.033333        -54.392593   

  06:30:0d:5d:fe:04 00:30:0d:5d:fe:04 da:9c:67:5f:b0:48 24:4b:fe:24:25:70  
1        -55.571296        -55.808333        -56.262037        -56.536111  

[1 rows x 22 columns]
   posx posy 06:30:0d:62:21:26 00:30:0d:62:21:26 00:40:5a:bf:d0:b1  \
1  1.35  -40        -43.441176        -43.563725        -46.708824   

  00:40:5a:bf:d0:b3 00:40:5a:bf:d0:b2 88:36:6c:c1:bf:96 70:5d:cc:80:73:80  \
1        -47.687255        

<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-input-12-aff822e869b0>:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  make_df = make_df.append({
<ipython-i

In [ ]:
# =================================================================================================================

def save():
  save_input_folder_path = path + 'Pre-processing ' + ver + '/pos/구간2' + '/MAC ' + str(nom) + '/temp/'
  save_output_folder_path = path + 'Pre-processing ' + ver + '/pos/구간2' + '/MAC ' + str(nom) + '/'

  # Create the output folder if it doesn't exist
  if not os.path.exists(save_output_folder_path):
      os.makedirs(save_output_folder_path)

  save_output_file_path = save_output_folder_path + temp + data_type + '_MAC ' + str(nom) + "_" + ver + ".csv"

  dfs = []
  for filename in os.listdir(save_input_folder_path):
      if filename.endswith('.csv'):
          # CSV 파일 읽기
          with open(os.path.join(save_input_folder_path, filename), 'r', encoding='UTF8') as f:
              make_df = pd.read_csv(f)
              dfs.append(make_df)

  # 모든 CSV 파일을 하나로 합치기
  merged_df = pd.concat(dfs, axis=0, ignore_index=True)

  # 각 열의 빈 행의 개수 카운트
  null_counts = merged_df.isnull().sum()

  # 빈 행의 개수가 5개 이상인 열 삭제
  cols_to_drop = null_counts[null_counts > 5].index
  merged2_df = merged_df.drop(cols_to_drop, axis=1)

  # 하나로 합쳐진 데이터를 CSV 파일로 저장. 인덱스 미포함
  merged2_df.to_csv(save_output_file_path, index=False, encoding='UTF8')

  print("save_input_folder_path: ", save_input_folder_path)
  print("save_output_folder_path: ", save_output_folder_path, end="\n\n")

# =================================================================================================================

for nom in nom_values:
  save()

save_input_folder_path:  /content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/230916 8th/Offline Data/Pre-processing v005/pos/구간2/MAC 2/temp/
save_output_folder_path:  /content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/230916 8th/Offline Data/Pre-processing v005/pos/구간2/MAC 2/

save_input_folder_path:  /content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/230916 8th/Offline Data/Pre-processing v005/pos/구간2/MAC 3/temp/
save_output_folder_path:  /content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/230916 8th/Offline Data/Pre-processing v005/pos/구간2/MAC 3/

save_input_folder_path:  /content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/230916 8th/Offline Data/Pre-processing v005/pos/구간2/MAC 4/temp/
save_output_folder_path:  /content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/230916 8th/Offline Data/Pre-processing v005/pos/구간2/MAC 4/

save_input_folder_path:  /content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/230916 8th/Offline Data/Pre-processing v005/pos/구간2

## Wi-Fi 수집 데이터 전처리 (1개) - 코드 2개

In [10]:
#input_folder_path = path + 'Original/'  # 필터 안 쓴 Original 데이터
input_folder_path = path + 'Moving Avg Filter_BF ' + buffer + '/Filtered Data/'  # 이동평균필터 적용한 데이터
output_folder_path = path + 'Pre-processing ' + ver + '/MAC ' + str(nom) + '/temp/'

print("input_folder_path: ", input_folder_path)
print("output_folder_path: ", output_folder_path, end="\n\n")

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

def process_csv_folder(input_folder_path, output_folder_path, sep=','):
    # Get a list of all files in the input folder
    file_list = os.listdir(input_folder_path)

    # Iterate over each file in the folder
    for file_name in file_list:
      print(file_name)
      # Construct the input and output file paths
      input_file_path = os.path.join(input_folder_path, file_name)
      output_file_name = os.path.splitext(file_name)[0] + "_MAC " + str(nom) + ".csv"
      output_file_path = os.path.join(output_folder_path, output_file_name)

      file_name = file_name.replace(".csv", "")

      # 파일 이름에서 알파벳 제거 / posx, posy 값을 추출
      pattern = re.compile('[a-zA-Z]')
      # 정규 표현식에 맞는 문자를 공백으로 대체하여 알파벳 문자를 제거
      file_name = re.sub(pattern, '', file_name)

      #posx, posy = re.findall(r'\d+', file_name)[0], re.findall(r'\d+', file_name)[1]
      #posx, posy = file_name.split('.csv')[0].split(',')  # 소수점
      posx, posy = file_name.split(',')  # 소수점

      # Process the CSV file and save the modified output file
      process_csv(input_file_path, output_file_path, posx, posy, sep)

def process_csv(input_file_path, output_file_path, x, y, sep=','):
    df = pd.read_csv(input_file_path, sep=sep)

    # Extract the MAC addresses from the first row and remove extra decimal points
    mac_addresses = df.columns.str.replace(r'\.\d+', '')

    # Calculate the mean values for each column
    mean_row = df.mean()

    # Create a new DataFrame with the MAC addresses and the mean values
    new_df = pd.DataFrame(columns=mac_addresses)
    new_df.loc[0] = mac_addresses
    new_df.loc[1] = mean_row.values

    # 'Timestamp, BT, BY, BZ, BX' 열을 삭제
    #new_df = new_df.iloc[:, :-5]

    # 평균 낸 RSS 값 선택
    rss_mean = new_df.iloc[1]

    # 평균 낸 RSS 값이 큰 순서로 데이터프레임 정렬
    sorted_df = new_df[rss_mean.sort_values(ascending=False).index]

    # 중복되는 행 삭제
    sorted_df = sorted_df.drop(0)

    # 설정한 AP 개수대로 신호 세기가 센 신호 남기기
    signal_df = sorted_df.iloc[:, 0:nom]

    # 첫 번째 열에 posx, posy 값을 추가
    posx = x
    posy = y
    signal_df.insert(0, 'posx', posx)
    signal_df.insert(1, 'posy', posy)

    make_df = pd.DataFrame(columns=["posx", "posy", "mac", "rss"])

    x = signal_df.iloc[0]["posx"]
    y = signal_df.iloc[0]["posy"]

    # Loop through each MAC address and RSS value
    for j in range(2, nom+2):
      try:
        # Get the MAC address and RSS value
        mac = signal_df.columns[j]
        rss = signal_df.iloc[0][j]
      except:
        print("nom: %d 오류" % nom)

      # Add a new row to the new DataFrame
      make_df = make_df.append({
          "posx": x,
          "posy": y,
          "mac": mac,
          "rss": rss
      }, ignore_index=True)

    make_df.to_csv(output_file_path, sep=sep, index=False, header=True)

# Process the CSV files in the input folder and save the modified output files
process_csv_folder(input_folder_path, output_folder_path)

input_folder_path:  /content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/230916 8th/Offline Data/Moving Avg Filter_BF 5/Filtered Data/
output_folder_path:  /content/drive/MyDrive/2023_CD/[자료] Wi-Fi/Collected Data/230916 8th/Offline Data/Pre-processing v005/MAC 20/temp/



FileNotFoundError: ignored

In [ ]:
input_folder_path = path + 'Pre-processing ' + ver + '/MAC ' + str(nom) + '/temp/'
output_folder_path = path + 'Pre-processing ' + ver + '/MAC ' + str(nom) + '/'

print("input_folder_path: ", input_folder_path)
print("output_folder_path: ", output_folder_path, end="\n\n")

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

output_file_path = output_folder_path + temp + data_type + '_MAC ' + str(nom) + "_" + ver + ".csv"

dfs = []
for filename in os.listdir(input_folder_path):
    if filename.endswith('.csv'):
        # CSV 파일 읽기
        with open(os.path.join(input_folder_path, filename), 'r', encoding='UTF8') as f:
            make_df = pd.read_csv(f)
            dfs.append(make_df)

# 모든 CSV 파일을 하나로 합치기
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

# 각 열의 빈 행의 개수 카운트
null_counts = merged_df.isnull().sum()

# 빈 행의 개수가 5개 이상인 열 삭제
cols_to_drop = null_counts[null_counts > 5].index
merged2_df = merged_df.drop(cols_to_drop, axis=1)

# 하나로 합쳐진 데이터를 CSV 파일로 저장. 인덱스 미포함
merged2_df.to_csv(output_file_path, index=False, encoding='UTF8')